This notebook will walk through training a simple single-stage object detector from scratch using
the `detection-utils` library. The detector you'll build here is based on the recent [RetinaNet in
the paper "Focal Loss for Dense Object Detection"](https://arxiv.org/abs/1708.02002). This notebook
will assume high-level familiarity with the problem of object detection and is not intended to be a
full tutorial for the problem. Rather, this is intended to be a full working example of using
`detection-utils` to construct an object detector. That said, a good amount of exposition is
provided, which can serve as an accelerated introduction to object detection.

The first step is to generate a dataset. We'll construct a simple dataset of 2d shapes on a
background of colored noise by using the `generate-dataset.py` script in this directory. You can run
this from the command line or just execute the cell below to generate 1000 training and 200
validation images. Feel free to tweak those numbers. Basic usage of the script is:

``` shell
$ python generate-dataset.py -d /path/to/destination -n number_of_images
```

For additional options, see `python generate-dataset.py -h` to get help output. If this takes more
than about 20-30 seconds, kill it and restart. The script is a bit brittle.

In [ ]:
!python generate-dataset.py -d data/train -n 1000
!python generate-dataset.py -d data/val -n 200

You should now have 1000 training and 200 validation images stored in `./data/train` and
`./data/val`, respectively. Each of the images will have between 3 and 8 objects per image and each
of the objects will be between 24 and 40 pixels in each dimension. Each of the images is 256x256 and
stored as RGB. We have three data files for each of our training and validation sets:

- `images.npy` is a `float32` array of shape `(N, 256, 256, 3)`, where `N` is the number of images.
- `boxes.npy` is an `object` array of shape `(N,)`. Each of the elements is an `int32` array of 
   shape `(K, 4)`, where `K` is the number of objects in that image.
- `labels.npy`is an `object` array of shape `(N,)`. Each of the elements is an `int32` array of
  shape `(K,)`, where `K` is the
- number of objects in that image.

Our `boxes` array stores boxes in `(left, top, right, bottom)` format, which we refer to as
`xyxy`. We'll discuss the data in more detail when we get to it. For now, let's cover our basic
imports.

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F

from detection_utils.boxes import non_max_suppression, generate_targets
from detection_utils.metrics import compute_recall, compute_precision
from detection_utils.pytorch import softmax_focal_loss

We'll start out by defining a few constants. First up is our batch size. We'll use a batch size of
16 here. Feel free to change this as needed for GPU memory or as desired. Choice of batch size is
not too important for training.

We'll also set the device to use for training. Valid options here are `'cuda'`, `'cuda:N'` where `N`
is the index of the GPU on your machine that you want to use, and `'cpu'`. If you have a GPU
available, we certainly recommend utilizing it. However, training this simple model on CPU is
possible; it won't take *too* much time to get a decent level of performance. If you have multiple
GPUs, we recommend choosing a GPU that you are not rendering your display on. Using your display GPU
for compute can cause some lockups and rendering issues/delays. Below, we would ordinarily change
`'cuda:0'` to a non-zero GPU, but we chose 0 to work out of the box for more people.

In [ ]:
batch_size = 16  # feel free to manipulate this as needed for GPU memory or as desired

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
# here we'll define a simple helper function to load a dataset
# then load our training and validation sets
def load_data(root_dir):
    root_dir = Path(root_dir)
    images = np.load(root_dir / 'images.npy')
    # we're loading object arrays here so we need to allow_pickle
    boxes = np.load(root_dir / 'boxes.npy', allow_pickle=True)
    labels = np.load(root_dir / 'labels.npy', allow_pickle=True)
    return images, boxes, labels

train_images, train_boxes, train_labels = load_data('data/train/')
val_images, val_boxes, val_labels = load_data('data/val/')

# for performance, we'll load all of our images onto our device ahead of time to minimize
# data transfers from CPU -> GPU. We know we're working with a small dataset, so this shouldn't
# cause problems. This uses about a gigabyte of GPU memory.
# We'll do the transpose to convert our images from shape (N, 256, 256, 3) to (N, 3, 256, 256)
# which is the ordering we need for our CNN.
train_images = torch.tensor(train_images.transpose(0, 3, 1, 2)).to(device)
val_images = torch.tensor(val_images.transpose(0, 3, 1, 2)).to(device)

Now we'll create our model. As mentioned previously, we're using an architecture based on
RetinaNet. Using a feature pyramid network would be incredible overkill, so we don't incorporate an
FPN. We end up with a very simple backbone network followed by simple classification and regression
heads. Feel free to tweak the network.

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, padding=1)
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1)
        self.conv3 = nn.Conv2d(20, 30, 3, padding=1)
        self.conv4 = nn.Conv2d(30, 40, 3, padding=1)
        
        self.classification = nn.Conv2d(40, 4, 1) # background / rectangle / triangle / circle
        self.regression = nn.Conv2d(40, 4, 1)
        
        for layer in (self.conv1, self.conv2, self.conv3, self.conv4,
                     self.classification, self.regression):
            nn.init.xavier_normal_(layer.weight, np.sqrt(2))
            nn.init.constant_(layer.bias, 0)

        nn.init.constant_(self.classification.bias[0], -4.6)  # rougly -log((1-π)/π) for π = 0.01
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        
        classifications = self.classification(x).permute(0, 2, 3, 1)                          # (N, R, C, # classes)
        classifications = classifications.reshape(x.shape[0], -1, classifications.shape[-1])  # (N, R*C, # classes)
        regressions = self.regression(x).permute(0, 2, 3, 1)                                  # (N, R, C, # classes)
        regressions = regressions.reshape(x.shape[0], -1, 4)                                  # (N, R*C, 4)
        return classifications, regressions

Next we'll need a set of anchor boxes for our detector. We know a priori that our objects are sized
in [24, 40] so we can pick an anchor box size somewhere around there. We'll go right in the middle
with an anchor box size of 32x32. This is a bit of a cheat, but simplifies the example and is
comparable to the common practice of clustering the box sizes in your training dataset. Our model
downsamples the image by a factor of 16, so we'll stride our anchors across the images with a step
of 16. This ensures that each entry in the feature map corresponds to the center of an anchor box 
in pixel space.

In [ ]:
anchor_boxes = []
for y in range(0, train_images.shape[2], 16):
    for x in range(0, train_images.shape[3], 16):
        anchor_boxes.append(np.array([-16, -16, 16, 16]) + np.array([x, y, x, y]))
anchor_boxes = np.vstack(anchor_boxes)
# anchor_boxes are in xyxy format

Now that we have our anchor boxes, we need to generate a set of targets for training. What we're
computing here is which anchor box(es) are responsible for each object in the training set. For each
anchor box responsible for an object, we determine the regression offsets needed to shift the anchor
to cover the object. We'll also figure out the corresponding label.

The real benefit of this is that all of our target lists are going to be the same shape (because the
number of anchor boxes is the same) regardless of how many objects are in each scene. This allows us
to batch processing, which we wouldn't be able to do otherwise.

In [ ]:
train_cls_targs, train_reg_targs = zip(*(generate_targets(anchor_boxes, bxs, lbls, 0.2, 0.1)
                                         for bxs, lbls in zip(train_boxes, train_labels)))
val_cls_targs, val_reg_targs = zip(*(generate_targets(anchor_boxes, bxs, lbls, 0.2, 0.1) 
                                     for bxs, lbls in zip(val_boxes, val_labels)))

# For convenience (and to minimize data transfers) we'll shove all of this onto our
# device as well. This only takes a couple of megabytes of GPU memory
train_reg_targs = torch.tensor(train_reg_targs).float().to(device)
train_cls_targs = torch.tensor(train_cls_targs).long().to(device)
val_reg_targs = torch.tensor(val_reg_targs).float().to(device)
val_cls_targs = torch.tensor(val_cls_targs).long().to(device)

In [ ]:
# our loss function is a combination of focal loss for classification
# and smooth L1 (Huber) loss for regression
def loss(class_predictions, regression_predictions, class_targets, regression_targets):
    class_targets = class_targets.reshape(-1)
    regression_targets = regression_targets.reshape(-1, 4)
    class_predictions = class_predictions.reshape(-1, class_predictions.shape[-1])
    regression_predictions = regression_predictions.reshape(-1, 4)

    regression_loss = torch.tensor(0).float()
    mask = torch.squeeze(class_targets > 0)
    num_foreground = mask.sum().item()
    if mask.numel() > 0:
        regression_loss = F.smooth_l1_loss(
            regression_predictions[mask], regression_targets[mask]
        )

    mask = torch.squeeze(class_targets > -1)
    
    # the sum of focal loss terms is normalized by the number
    # of anchors assigned to a ground-truth box
    classification_loss = softmax_focal_loss(
        class_predictions[mask],
        class_targets[mask],
        alpha=0.25,
        gamma=2,
        reduction="sum",
    ) / num_foreground

    return classification_loss, regression_loss

In [ ]:
def compute_detections(classifications, regressions, feature_map_width, anchor_box_step, anchor_box_size):
    """ Compute a set of boxes, class predictions, and foreground scores from
        detection model outputs.

    Parameters
    ----------
    classifications : torch.Tensor, shape=(N, R*C, # classes)
        A set of class predictions at each spatial location.

    regressions : torch.Tensor, shape=(N, R*C, 4)
        A set of predicted box offsets, in (x, y, w, h) at each spatial location.

    feature_map_width : int
        The number of pixels in the feature map, along the x direction.

    anchor_box_step : int
        The number of pixels (in image space) between each anchor box.

    anchor_box_size : int
        The side length of the anchor box.

    Returns
    -------
    Tuple[numpy.ndarray shape=(R*C, 4), numpy.ndarray shape=(R*C, 1), numpy.ndarray shape=(R*C,)]
        The (boxes, class predictions, foreground scores) at each spatial location.
    """
    box_predictions = np.empty((len(regressions), 4), dtype=np.float32)
    scores = torch.softmax(classifications, dim=-1).detach().cpu().numpy()
    scores = 1 - scores[:, 0]  # foreground score

    class_predictions = classifications.argmax(dim=-1, keepdim=True).detach().cpu().numpy()
    regressions = regressions.detach().cpu().numpy()

    y, x = np.divmod(np.arange(len(classifications)), feature_map_width, dtype=np.float32)
    x_reg, y_reg, w_reg, h_reg = regressions.T  # transform (R*C, 4) to (4, R*C) for assignment
    x = anchor_box_step * x + anchor_box_size * x_reg
    y = anchor_box_step * y + anchor_box_size * y_reg

    half_w = np.clip(np.exp(w_reg), 0, 10**6) * anchor_box_size / 2
    half_h = np.clip(np.exp(h_reg), 0, 10**6) * anchor_box_size / 2

    box_predictions[:, 0] = x - half_w  # x1
    box_predictions[:, 1] = y - half_h  # y1
    box_predictions[:, 2] = x + half_w  # x2
    box_predictions[:, 3] = y + half_h  # y2

    return box_predictions, class_predictions, scores

In [ ]:
def compute_batch_stats(class_predictions, regression_predictions, boxes, labels, feature_map_width,
                        anchor_box_step=16, anchor_box_size=32, threshold=0.5):
    """ Compute the batch statistics (AP and AR) given a batch of predictions and truth.

    Parameters
    ----------
    class_predictions : numpy.ndarray, shape=(N, K, C)
        The predicted class scores of each of N images at each of K anchor boxes.

    regression_predictions : numpy.ndarray, shape=(N, K, 4)
        The predicted regression values of each of N images at each of K anchor boxes.

    boxes : numpy.ndarray, shape=(N,)
        The truth boxes for each image. Note that each of the N elements is of 
        shape (W_i, 4), where W_i is the number of objects in image i.

    labels : numpy.ndarray, shape=(N,)
        The truth labels for each image. Note that each of the N elements is of
        shape (W_i,), where  W_i is the number of objects in image i.

    feature_map_width : int, optional (default=40)
        The width of the feature map.

    anchor_box_step : int, optional (default=16)
        The stride across the image at which anchor boxes are placed.

    anchor_box_size : int, optional (default=32)
        The side length of each anchor box.

    threshold : Real, optional (default=0.5)
        The confidence threshold under which to cull predictions.

    Returns
    -------
    Tuple[List[float], List[float]]
        The (aps, ars) for the images.
    """
    aps, ars = [], []
    for i in range(len(class_predictions)):
        truth_detections = np.hstack((boxes[i], labels[i][:, None]))

        box_preds, class_preds, scores = compute_detections(class_predictions[i], 
                                                            regression_predictions[i],
                                                            feature_map_width, 
                                                            anchor_box_step, 
                                                            anchor_box_size)

        keep_idxs = non_max_suppression(box_preds, scores, 0.3)
        detections = np.hstack((box_preds, class_preds))
        detections = detections[keep_idxs]
        detections = detections[scores[keep_idxs] > threshold]

        aps.append(compute_precision(detections, truth_detections, 0.5))
        ars.append(compute_recall(detections, truth_detections, 0.5))
    return aps, ars

Now we can create our model and train!

In [ ]:
model = Model().to(device)
optim = torch.optim.Adam(model.parameters(), weight_decay=5e-04)

In [ ]:
def train_epoch(epoch, batch_size, log_file=None, eval_every=10):
    torch.set_grad_enabled(True)
    model.train()

    idxs = torch.randperm(len(train_images))  # shuffle indices
    for itr, batch in enumerate(
        (slice(i, i + batch_size) for i in range(0, len(train_images), batch_size))
    ):
        batch = idxs[batch]

        class_predictions, regression_predictions = model(train_images[batch])
        aps, ars = compute_batch_stats(
            class_predictions,
            regression_predictions,
            train_boxes[batch],
            train_labels[batch],
            feature_map_width=16,
        )
        total_cls_loss, total_reg_loss = loss(
            class_predictions,
            regression_predictions,
            train_cls_targs[batch],
            train_reg_targs[batch],
        )

        optim.zero_grad()
        (total_cls_loss + total_reg_loss).backward()
        optim.step()


        if log_file is not None:
            log_str = f"Epoch {epoch:03d} batch {itr+1:05d}: "
            log_str += f"Loss {(total_cls_loss + total_reg_loss).item():0.4f}"
            log_str += f" Regression loss {total_reg_loss.item():0.4f}"
            log_str += f" Classification loss {total_cls_loss.item():0.4f}"
            log_str += f" AP {np.mean(aps):0.4f} AR {np.mean(ars):0.4f}"
            log_file.write(log_str + "\n")

        if (itr + 1) % 10 == 0:
            print(f"Epoch {epoch} iteration {itr + 1}", end="\r")

    if epoch % eval_every == 0:
        aps, ars = [], []
        idxs = torch.arange(len(val_images))
        for itr, batch in enumerate(
            (slice(i, i + batch_size) for i in range(0, len(val_images), batch_size))
        ):
            batch = idxs[batch]

            class_predictions, regression_predictions = model(val_images[batch])
            ap, ar = compute_batch_stats(
                class_predictions,
                regression_predictions,
                val_boxes[batch],
                val_labels[batch],
                feature_map_width=16,
            )
            aps += ap
            ars += ar

        print(
            f"Epoch {epoch:03d}  Validation: {np.mean(aps):0.3f} mAP, {np.mean(ars):0.3f} mAR"
        )

In [ ]:
with open('log.txt', 'w') as log:
    for epoch in range(1, 101):
        train_epoch(epoch, batch_size, log, eval_every=20)

At this point, we tend to get around 65% precision and 50% recall. We'll go through a visualization
below, but note that you can keep training for longer to get better performance. We've found that
going for 500 epochs tends to do pretty well. You can also generate more training samples at the
top of the notebook and keep the number of epochs the same. They key is additional training
iterations. After 500 epochs we're up to about 75% precision and 75% recall.

Now we can visualize the training process and see how our model is doing.

In [ ]:
losses, aps, ars = [], [], []
with open('log.txt') as f:
    for line in f:
        line = line.split()
        losses.append(float(line[5]))
        aps.append(float(line[-3]))
        ars.append(float(line[-1]))

fig, ax = plt.subplots()
ax.scatter(range(len(aps)), aps)
ax.scatter(range(len(ars)), ars)

In [ ]:
def visualize_random_image(model, threshold=0, idx=None):
    fig, ax = plt.subplots(figsize=(10, 10))
    idx = np.random.randint(len(val_images)) if idx is None else idx
    print(idx)
    img = val_images[idx]
    ax.imshow(img.permute(1, 2, 0).cpu())
    ax.axis('off')

    out_cls, out_reg = model(img[np.newaxis])
    box_preds, class_preds, scores = compute_detections(out_cls.squeeze(), 
                                                        out_reg.squeeze(), 
                                                        feature_map_width=16,
                                                        anchor_box_step=16, 
                                                        anchor_box_size=32)

    keep = scores > threshold
    box_preds = box_preds[keep]
    class_preds = class_preds[keep]
    scores = scores[keep]
    keep_idxs = non_max_suppression(box_preds, scores, threshold=0.1)
    box_preds = box_preds[keep_idxs]
    class_preds = class_preds[keep_idxs]

    for class_pred, box_pred in zip(class_preds, box_preds):
        if class_pred > 0:
            x1, y1, x2, y2 = box_pred
            ax.add_patch(Rectangle((x1, y1), x2 - x1, y2 - y1, color='r', fill=None, lw=2))
            label = int(class_pred)
            label = 'rectangle' if label == 1 else label
            label = 'triangle' if label == 2 else label
            label = 'circle' if label == 3 else label
            ax.annotate(label, (x1, y1), color='r', fontsize=24)

In [ ]:
visualize_random_image(model, idx=None)